In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
import webbrowser
import os
from google.colab import files

In [3]:
# Download NLTK lexicon
nltk.download('vader_lexicon')

# Upload both CSV files
uploaded = files.upload()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Saving Play Store Data.csv to Play Store Data.csv
Saving User Reviews.csv to User Reviews.csv


In [4]:
# Load datasets
apps_df = pd.read_csv('Play Store Data.csv')
reviews_df = pd.read_csv('User Reviews.csv')


In [5]:
# Data cleaning
apps_df = apps_df.dropna(subset=['Rating'])
for column in apps_df.columns:
    apps_df[column].fillna(apps_df[column].mode()[0], inplace=True)
apps_df.drop_duplicates(inplace=True)
apps_df = apps_df[apps_df['Rating'] <= 5]
reviews_df.dropna(subset=['Translated_Review'], inplace=True)

apps_df['Installs'] = apps_df['Installs'].str.replace(',', '').str.replace('+', '').astype(int)
apps_df['Price'] = apps_df['Price'].str.replace('$', '').astype(float)


/tmp/ipython-input-1847050285.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  apps_df[column].fillna(apps_df[column].mode()[0], inplace=True)


In [6]:
# Merge datasets
merged_df = pd.merge(apps_df, reviews_df, on='App', how='inner')


In [7]:
# Convert size
def convert_size(size):
    if 'M' in size:
        return float(size.replace('M', ''))
    elif 'k' in size:
        return float(size.replace('k', '')) / 1024
    else:
        return np.nan
apps_df['Size'] = apps_df['Size'].apply(convert_size)

In [8]:
# Log transforms
apps_df['Log_Installs'] = np.log(apps_df['Installs'])
apps_df['Reviews'] = apps_df['Reviews'].astype(int)
apps_df['Log_Reviews'] = np.log(apps_df['Reviews'])

In [9]:
# Rating group
def rating_group(rating):
    if rating >= 4:
        return 'Top rated app'
    elif rating >= 3:
        return 'Above average'
    elif rating >= 2:
        return 'Average'
    else:
        return 'Below Average'
apps_df['Rating_Group'] = apps_df['Rating'].apply(rating_group)

In [10]:
# Revenue
apps_df['Revenue'] = apps_df['Price'] * apps_df['Installs']

# Sentiment analysis
sia = SentimentIntensityAnalyzer()
reviews_df['Sentiment_Score'] = reviews_df['Translated_Review'].apply(lambda x: sia.polarity_scores(str(x))['compound'])

# Date conversion
apps_df['Last Updated'] = pd.to_datetime(apps_df['Last Updated'], errors='coerce')
apps_df['Year'] = apps_df['Last Updated'].dt.year

# Dashboard setup
html_files_path = "./"
os.makedirs(html_files_path, exist_ok=True)
plot_containers = ""

In [11]:
def save_plot_as_html(fig, filename, insight):
    global plot_containers
    filepath = os.path.join(html_files_path, filename)
    html_content = pio.to_html(fig, full_html=False, include_plotlyjs='inline')
    plot_containers += f"""
    <div class="plot-container" id="{filename}" onclick="openPlot('{filename}')">
        <div class="plot">{html_content}</div>
        <div class="insights">{insight}</div>
    </div>
    """
    fig.write_html(filepath, full_html=False, include_plotlyjs='inline')
# Style
plot_width = 400
plot_height = 300
plot_bg_color = 'black'
text_color = 'white'
title_font = {'size': 16}
axis_font = {'size': 12}

In [12]:
# Plot 1: Category Distribution
category_counts = apps_df['Category'].value_counts().nlargest(10)
fig1 = px.bar(x=category_counts.index, y=category_counts.values, labels={'x':'Category','y':'Count'},
              title='Top Categories on Play Store', color=category_counts.index,
              color_discrete_sequence=px.colors.sequential.Plasma, width=plot_width, height=plot_height)
fig1.update_layout(plot_bgcolor=plot_bg_color, paper_bgcolor=plot_bg_color, font_color=text_color,
                   title_font=title_font, xaxis=dict(title_font=axis_font), yaxis=dict(title_font=axis_font),
                   margin=dict(l=10,r=10,t=30,b=10))
save_plot_as_html(fig1, "Category Graph 1.html", "Top categories: Tools, Entertainment, Productivity")

In [13]:
# Plot 2: Type Distribution
type_counts = apps_df['Type'].value_counts()
fig2 = px.pie(values=type_counts.values, names=type_counts.index, title='App Type Distribution',
              color_discrete_sequence=px.colors.sequential.RdBu, width=plot_width, height=plot_height)
fig2.update_layout(plot_bgcolor=plot_bg_color, paper_bgcolor=plot_bg_color, font_color=text_color,
                   title_font=title_font, margin=dict(l=10,r=10,t=30,b=10))
save_plot_as_html(fig2, "Type Graph 2.html", "Most apps are Free, monetized via ads or purchases")


In [14]:
# Plot 3: Rating Histogram
fig3 = px.histogram(apps_df, x='Rating', nbins=20, title='Rating Distribution',
                   color_discrete_sequence=['#636EFA'], width=plot_width, height=plot_height)
fig3.update_layout(plot_bgcolor=plot_bg_color, paper_bgcolor=plot_bg_color, font_color=text_color,
                   title_font=title_font, xaxis=dict(title_font=axis_font), yaxis=dict(title_font=axis_font),
                   margin=dict(l=10,r=10,t=30,b=10))
save_plot_as_html(fig3, "Rating Graph 3.html", "Ratings are skewed positively")

In [15]:
# Plot 4: Sentiment Distribution
sentiment_counts = reviews_df['Sentiment_Score'].value_counts()
fig4 = px.bar(x=sentiment_counts.index, y=sentiment_counts.values,
              labels={'x':'Sentiment Score','y':'Count'}, title='Sentiment Distribution',
              color=sentiment_counts.index, color_discrete_sequence=px.colors.sequential.RdPu,
              width=plot_width, height=plot_height)
fig4.update_layout(plot_bgcolor=plot_bg_color, paper_bgcolor=plot_bg_color, font_color=text_color,
                   title_font=title_font, xaxis=dict(title_font=axis_font), yaxis=dict(title_font=axis_font),
                   margin=dict(l=10,r=10,t=30,b=10))
save_plot_as_html(fig4, "Sentiment Graph 4.html", "Most reviews lean slightly positive")

In [16]:
# Plot 5: Installs by Category
installs_by_category = apps_df.groupby('Category')['Installs'].sum().nlargest(10)
fig5 = px.bar(x=installs_by_category.index, y=installs_by_category.values,
              labels={'x':'Category','y':'Installs'}, title='Installs by Category',
              color=installs_by_category.index, color_discrete_sequence=px.colors.sequential.Blues,
              width=plot_width, height=plot_height)
fig5.update_layout(plot_bgcolor=plot_bg_color, paper_bgcolor=plot_bg_color, font_color=text_color,
                   title_font=title_font, xaxis=dict(title_font=axis_font), yaxis=dict(title_font=axis_font),
                   margin=dict(l=10,r=10,t=30,b=10))
save_plot_as_html(fig5, "Installs Graph 5.html", "Communication & Social apps lead in installs")

In [17]:
# Plot 6: Updates per Year
updates_per_year = apps_df['Last Updated'].dt.year.value_counts().sort_index()
fig6 = px.line(x=updates_per_year.index, y=updates_per_year.values,
               labels={'x': 'Year', 'y': 'Number of Updates'}, title='Number of Updates Over the Years',
               color_discrete_sequence=['#AB63FA'], width=plot_width, height=plot_height)
fig6.update_layout(plot_bgcolor=plot_bg_color, paper_bgcolor=plot_bg_color, font_color=text_color,
                   title_font=title_font, xaxis=dict(title_font=axis_font), yaxis=dict(title_font=axis_font),
                   margin=dict(l=10, r=10, t=30, b=10))
save_plot_as_html(fig6, "Updates Graph 6.html", "Updates show active maintenance by developers")

In [18]:
# Plot 7: Revenue by Category
revenue_by_category = apps_df.groupby('Category')['Revenue'].sum().nlargest(10)
fig7 = px.bar(x=revenue_by_category.index, y=revenue_by_category.values,
              labels={'x':'Category','y':'Revenue'}, title='Revenue by Category',
              color=revenue_by_category.index, color_discrete_sequence=px.colors.sequential.Greens,
              width=plot_width, height=plot_height)
fig7.update_layout(plot_bgcolor=plot_bg_color, paper_bgcolor=plot_bg_color, font_color=text_color,
                   title_font=title_font, xaxis=dict(title_font=axis_font), yaxis=dict(title_font=axis_font),
                   margin=dict(l=10,r=10,t=30,b=10))
save_plot_as_html(fig7, "Revenue Graph 7.html", "Business & Productivity apps earn the most")

In [19]:
# Plot 8: Top Genres
genre_counts = apps_df['Genres'].str.split(';', expand=True).stack().value_counts().nlargest(10)
fig8 = px.bar(x=genre_counts.index, y=genre_counts.values,
              labels={'x':'Genre','y':'Count'}, title='Top Genres',
              color=genre_counts.index, color_discrete_sequence=px.colors.sequential.OrRd,
              width=plot_width, height=plot_height)
fig8.update_layout(plot_bgcolor=plot_bg_color, paper_bgcolor=plot_bg_color, font_color=text_color,
                   title_font=title_font, xaxis=dict(title_font=axis_font), yaxis=dict(title_font=axis_font),
                   margin=dict(l=10,r=10,t=30,b=10))
save_plot_as_html(fig8, "Genre Graph 8.html", "Action and Casual genres dominate")

In [20]:
# Plot 9: Update Date vs Rating
fig9 = px.scatter(apps_df, x='Last Updated', y='Rating', color='Type',
                  title='Impact of Last Update on Rating',
                  color_discrete_sequence=px.colors.qualitative.Vivid,
                  width=plot_width, height=plot_height)
fig9.update_layout(plot_bgcolor=plot_bg_color, paper_bgcolor=plot_bg_color, font_color=text_color,
                   title_font=title_font, xaxis=dict(title_font=axis_font), yaxis=dict(title_font=axis_font),
                   margin=dict(l=10,r=10,t=30,b=10))
save_plot_as_html(fig9, "Update Graph 9.html", "Frequent updates don’t guarantee higher ratings")

In [21]:
# Plot 10: Ratings - Paid vs Free
fig10 = px.box(apps_df, x='Type', y='Rating', color='Type',
               title='Rating for Paid vs Free Apps',
               color_discrete_sequence=px.colors.qualitative.Pastel,
               width=plot_width, height=plot_height)
fig10.update_layout(plot_bgcolor=plot_bg_color, paper_bgcolor=plot_bg_color, font_color=text_color,
                    title_font=title_font, xaxis=dict(title_font=axis_font), yaxis=dict(title_font=axis_font),
                    margin=dict(l=10,r=10,t=30,b=10))
save_plot_as_html(fig10, "Paid Free Graph 10.html", "Paid apps tend to have better ratings")

In [22]:
# Final dashboard HTML
print("Number of plots:", plot_containers.count('plot-container'))

dashboard_html = f"""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width,initial-scale=1.0">
    <title>Google Play Store Review Analytics</title>
    <style>
        body {{ font-family: Arial, sans-serif; background-color: #333; color: #fff; margin: 0; padding: 0; }}
        .header {{ display: flex; align-items: center; justify-content: center; padding: 20px; background-color: #444 }}
        .header img {{ margin: 0 10px; height: 50px; }}
        .container {{ display: flex; flex-wrap: wrap; justify-content: center; padding: 20px; }}
        .plot-container {{ border: 2px solid #555; margin: 10px; padding: 10px; width: {plot_width}px; height: {plot_height}px; overflow: hidden; position: relative; cursor: pointer; }}
        .insights {{ display: none; position: absolute; right: 10px; top: 10px; background-color: rgba(0,0,0,0.7); padding: 5px; border-radius: 5px; color: #fff; }}
        .plot-container:hover .insights {{ display: block; }}
    </style>
    <script>
        function openPlot(filename) {{ window.open(filename, '_blank'); }}
    </script>
</head>
<body>
    <div class="header">
        <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/4/4a/Logo_2013_Google.png/800px-Logo_2013_Google.png" alt="Google Logo">
        <h1>Google Play Store Reviews Analytics</h1>
        <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/7/78/Google_Play_Store_badge_EN.svg/1024px-Google_Play_Store_badge_EN.svg.png" alt="Google Play Store Logo">
    </div>
    <div class="container">
        {plot_containers}
    </div>
</body>
</html>
"""

dashboard_path = os.path.join(html_files_path, "web_page.html")
with open(dashboard_path, "w", encoding="utf-8") as f:
    f.write(dashboard_html)

# Launch in browser
webbrowser.open('file://' + os.path.realpath(dashboard_path))

Number of plots: 30


False

In [23]:
from google.colab import files

# Download the dashboard HTML file
files.download(dashboard_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Assinged tasks

In [24]:
import pandas as pd
import plotly.express as px
from datetime import datetime
import pytz

# Helper: current IST time
ist = pytz.timezone('Asia/Kolkata')
now = datetime.now(ist)
hour = now.hour
print("Current IST Hour:", hour)


Current IST Hour: 15


In [28]:
if 15 <= hour < 17:  # 3PM to 5PM
    df1 = apps_df.copy()
    df1 = df1[(df1['Rating'] >= 4.0) &
              (df1['Size'] >= 10) &
              (df1['Last Updated'].dt.month == 1)]

    top10 = df1.groupby('Category')['Installs'].sum().nlargest(10).index
    df1 = df1[df1['Category'].isin(top10)]

    grouped = df1.groupby('Category').agg(
        avg_rating=('Rating','mean'),
        total_reviews=('Reviews','sum')
    ).reset_index()

    fig = px.bar(grouped, x='Category', y=['avg_rating','total_reviews'],
                 barmode='group', title="Avg Rating & Reviews by Top 10 Categories")
    fig.show()
else:
    print("Task 1 chart available only between 3PM-5PM IST")


In [30]:
if 18 <= hour < 20:  # 6PM to 8PM
    df2 = apps_df.copy()
    df2 = df2[~df2['Category'].str.startswith(('A','C','G','S'))]

    top5 = df2.groupby('Category')['Installs'].sum().nlargest(5).index
    df2 = df2[df2['Category'].isin(top5)]

    df2['highlight'] = df2['Installs'] > 1_000_000

    fig = px.choropleth(df2,
                        locations="Country",  # ⚠️ ensure you have a 'Country' column
                        color="Installs",
                        hover_name="Category",
                        title="Global Installs by Category")
    fig.show()
else:
    print("Task 2 chart available only between 6PM-8PM IST")


Task 2 chart available only between 6PM-8PM IST


In [31]:
if 13 <= hour < 14:  # 1PM to 2PM
    df3 = apps_df.copy()
    df3 = df3[(df3['Installs'] >= 10_000) &
              (df3['Revenue'] >= 10_000) &
              (df3['Android Ver'] > 4.0) &
              (df3['Size'] > 15) &
              (df3['Content Rating'] == 'Everyone') &
              (df3['App'].str.len() <= 30)]

    top3 = df3.groupby('Category')['Installs'].sum().nlargest(3).index
    df3 = df3[df3['Category'].isin(top3)]

    grouped = df3.groupby(['Category','Type']).agg(
        avg_installs=('Installs','mean'),
        avg_revenue=('Revenue','mean')
    ).reset_index()

    fig = px.bar(grouped, x='Category', y='avg_installs', color='Type',
                 title="Installs vs Revenue Free vs Paid", barmode='group')
    fig.add_scatter(x=grouped['Category'], y=grouped['avg_revenue'],
                    mode='lines+markers', name='Revenue', yaxis='y2')
    fig.update_layout(
        yaxis2=dict(overlaying='y', side='right', title='Revenue')
    )
    fig.show()
else:
    print("Task 3 chart available only between 1PM-2PM IST")


Task 3 chart available only between 1PM-2PM IST


In [32]:
if 18 <= hour < 21:  # 6PM to 9PM
    df4 = apps_df.copy()
    df4 = df4[(df4['Category'].str.startswith(('E','C','B'))) &
              (~df4['App'].str.startswith(('x','y','z'))) &
              (df4['Reviews'] > 500) &
              (~df4['App'].str.contains('S'))]

    # Translate categories
    translate = {
        'Beauty': 'सौंदर्य',   # Hindi
        'Business': 'வணிகம்', # Tamil
        'Dating': 'Partnersuche' # German
    }
    df4['Category'] = df4['Category'].replace(translate)

    monthly = df4.groupby([pd.Grouper(key='Last Updated', freq='M'),'Category'])['Installs'].sum().reset_index()
    monthly['pct_change'] = monthly.groupby('Category')['Installs'].pct_change()*100

    fig = px.line(monthly, x='Last Updated', y='Installs', color='Category',
                  title="Monthly Install Trends")
    # Shade where growth > 20%
    for cat in monthly['Category'].unique():
        sub = monthly[(monthly['Category']==cat) & (monthly['pct_change']>20)]
        fig.add_scatter(x=sub['Last Updated'], y=sub['Installs'],
                        fill='tozeroy', mode='none', name=f"{cat} growth>20%")
    fig.show()
else:
    print("Task 4 chart available only between 6PM-9PM IST")


Task 4 chart available only between 6PM-9PM IST


In [33]:
if 17 <= hour < 19:  # 5PM to 7PM
    df5 = apps_df.copy()
    valid_cats = ['Game','Beauty','Business','Comics','Communication','Dating','Entertainment','Social','Event']
    df5 = df5[(df5['Rating'] > 3.5) &
              (df5['Category'].isin(valid_cats)) &
              (df5['Reviews'] > 500) &
              (df5['Installs'] > 50_000) &
              (~df5['App'].str.contains('S')) &
              (df5['Sentiment_Subjectivity'] > 0.5)]

    translate = {
        'Beauty': 'सौंदर्य',
        'Business': 'வணிகம்',
        'Dating': 'Partnersuche'
    }
    df5['Category'] = df5['Category'].replace(translate)

    fig = px.scatter(df5, x='Size', y='Rating', size='Installs',
                     color='Category', title="Size vs Rating Bubble Chart")
    fig.update_traces(marker=dict(line=dict(width=1,color='DarkSlateGrey')))

    # Highlight Game category in pink
    game_mask = df5['Category'] == 'Game'
    fig.add_scatter(x=df5.loc[game_mask,'Size'],
                    y=df5.loc[game_mask,'Rating'],
                    mode='markers',
                    marker=dict(size=df5.loc[game_mask,'Installs']/100000,
                                color='pink'),
                    name='Game Highlight')
    fig.show()
else:
    print("Task 5 chart available only between 5PM-7PM IST")


Task 5 chart available only between 5PM-7PM IST
